In [36]:
import pandas as pd
import os
from os.path import join
import sys
from bokeh.io import show, output_notebook
from bokeh.charts import Scatter

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
from config import DATA_DIR

## Load Data

Download data to DATA_DIR from 
https://www.kaggle.com/ksuchris2000/oklahoma-earthquakes-and-saltwater-injection-wells/data

In [5]:
wells = pd.read_csv(join(DATA_DIR, 'InjectionWells.csv'))
wells.shape

(11126, 21)

In [6]:
wells.columns

Index(['API#', 'Operator', 'Operator ID', 'WellType', 'WellName', 'WellNumber',
       'OrderNumbers', 'Approval Date', 'County', 'Sec', 'Twp', 'Rng', 'QQQQ',
       'LAT', 'LONG', 'PSI', 'BBLS', 'ZONE', 'Unnamed: 18', 'Unnamed: 19',
       'Unnamed: 20'],
      dtype='object')

In [7]:
wells = wells.drop(columns=wells.columns[-3:])

In [50]:
wells = wells.dropna(axis=0)
wells.shape

(9656, 18)

In [8]:
def convert_to_float(x):
    """
    Converts a string to a float and returns null for values that don't match
    """
    try:
        return float(x.replace(',', ''))
    except Exception:
        return None
convert_to_float('1,550'), convert_to_float('ASDFL#_V@@R')

(1550.0, None)

In [9]:
def convert_series_to_float(pd_series):
    return pd_series.apply(convert_to_float)

In [10]:
wells.loc[:, 'PSI'] = wells['PSI'].str.replace(',','').apply(convert_to_float)

In [11]:
wells.loc[:, 'BBLS'] = convert_series_to_float(wells['BBLS'])

In [12]:
show(Scatter(wells, 'BBLS', 'PSI'))

In [13]:
wells.dtypes

API#             float64
Operator          object
Operator ID      float64
WellType          object
WellName          object
WellNumber        object
OrderNumbers     float64
Approval Date     object
County            object
Sec               object
Twp               object
Rng               object
QQQQ              object
LAT              float64
LONG             float64
PSI              float64
BBLS             float64
ZONE              object
dtype: object

In [14]:
float_types = wells.dtypes == float
float_types.values

array([ True, False,  True, False, False, False,  True, False, False,
       False, False, False, False,  True,  True,  True,  True, False])

In [15]:
from sklearn.preprocessing import LabelEncoder

In [16]:
from sklearn.linear_model import Ridge

In [17]:
r = Ridge()

In [18]:
y_col = 'BBLS'

In [73]:
wells_float = wells.select_dtypes(include='float').dropna(axis=0)

In [99]:
wells_float = wells_float.drop(y_col, axis=1)
x = wells_float

In [81]:
y = wells_float[y_col]

In [98]:
x.shape

(9656, 6)

In [23]:
assert x.shape[0] == y.shape[0]

In [24]:
x.columns

Index(['API#', 'Operator ID', 'OrderNumbers', 'LAT', 'LONG', 'PSI'], dtype='object')

In [25]:
x.head()

,API#,Operator ID,OrderNumbers,LAT,LONG,PSI
0,3.500300e+09,19499.0,133856.0,36.900324,-98.21826,2500.0
1,3.500300e+09,4030.0,470258.0,36.896636,-98.17772,2400.0
2,3.500321e+09,22182.0,329426.0,36.806113,-98.32584,1050.0
3,3.500321e+09,22281.0,281652.0,36.888589,-98.31853,3152.0
4,3.500321e+09,4030.0,470257.0,36.892128,-98.19462,1000.0


In [26]:
r.fit(x, y)

/home/bobdavis/.virtualenvs/machine-learning-examples/lib/python3.5/site-packages/scipy/linalg/basic.py:40: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number/precision: 3.742777922766492e-18 / 1.1102230246251565e-16
  RuntimeWarning)


Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [27]:
r.score(x, y)

0.13981939618988892

In [28]:
from sklearn.ensemble import RandomForestRegressor

In [29]:
rfr = RandomForestRegressor()

In [30]:
rfr.fit(x, y)
rfr.score(x, y)

0.9501910353213893

In [42]:
le = LabelEncoder()

In [51]:
wells_objects = wells.select_dtypes(exclude='float')
wells_objects.shape

(9656, 11)

In [52]:
wells_objects.head()

,Operator,WellType,WellName,WellNumber,Approval Date,County,Sec,Twp,Rng,QQQQ,ZONE
0,PHOENIX PETROCORP INC,2R,SE EUREKA UNIT-TUCKER #1,21,9/6/1977,ALFALFA,13,28N,10W,C-SE SE,CHEROKEE
1,CHAMPLIN EXPLORATION INC,2R,CHRISTENSEN,1,11/27/2002,ALFALFA,21,28N,09W,C-NW NW,RED FORK
2,LINN OPERATING INC,2R,NE CHEROKEE UNIT,85,8/19/1988,ALFALFA,24,27N,11W,SE NE,RED FORK
3,SANDRIDGE EXPLORATION & PRODUCTION LLC,2R,VELMA,2-19,7/11/1985,ALFALFA,19,28N,10W,SW NE NE SW,RED FORK
4,CHAMPLIN EXPLORATION INC,2R,GRAY,1A,11/27/2002,ALFALFA,20,28N,09W,SE SW NW,RED FORK


In [54]:
wells_objects.iloc[:, 0].sample(10)

2240       CITATION OIL & GAS CORPORATION
8314                              TAT INC
10939                   CHEW-COAST & SONS
10780         SAND BOY PRODUCTION COMPANY
7873                   ENERGYQUEST II LLC
151                        J-BREX COMPANY
805        CITATION OIL & GAS CORPORATION
9441     TARGA PIPELINE MID-CONTINENT LLC
3009         MID-CON ENERGY OPERATING LLC
4621              RED RIVER PETROLEUM INC
Name: Operator, dtype: object

In [55]:
wells_objects_encoded = le.fit_transform(wells_objects.iloc[:, 0])
wells_objects_encoded.shape

(9656,)

In [59]:
import numpy as np
np.random.choice(wells_objects_encoded, 30)

array([  82,  172, 1390, 1136,  257, 1073,  230,  260,  214,  257,  828,
        780,  668,  271,  764,  159, 1104,  786,  234,  264,  624,  739,
       1056,  624, 1198, 1114,  933,  224, 1043, 1149])

In [60]:
def encode_objects(pd_series):
    return le.fit_transform(pd_series)

In [69]:
x_objects = {}
for _, col in wells_objects.iteritems():
    x_objects[_] = encode_objects(col)

In [72]:
x_objects = pd.DataFrame.from_dict(x_objects)

In [85]:
x_objects.shape, wells_float.shape

((9656, 11), (9656, 7))

In [100]:
wells_encoded = x_objects.join(wells_float)
wells_encoded.shape

(9656, 17)

In [101]:
x = wells_encoded.dropna()
x.shape

(8263, 17)

In [102]:
y = y[x.index]

In [103]:
y.shape

(8263,)

In [104]:
x.columns

Index(['Approval Date', 'County', 'Operator', 'QQQQ', 'Rng', 'Sec', 'Twp',
       'WellName', 'WellNumber', 'WellType', 'ZONE', 'API#', 'Operator ID',
       'OrderNumbers', 'LAT', 'LONG', 'PSI'],
      dtype='object')

In [92]:
from sklearn.model_selection import train_test_split

In [105]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [106]:
r.fit(x_train, y_train)
train_score = r.score(x_train, y_train)
val_score = r.score(x_test, y_test)

/home/bobdavis/.virtualenvs/machine-learning-examples/lib/python3.5/site-packages/scipy/linalg/basic.py:40: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number/precision: 1.9330730826984756e-18 / 1.1102230246251565e-16
  RuntimeWarning)


In [107]:
train_score, val_score

(0.16279121593147838, 0.1749480909834079)

In [108]:
y_test_pred = r.predict(x_test)

In [109]:
show(Scatter(pd.DataFrame.from_dict({'y_pred': y_test_pred, 'y_actual': y_test}), 'y_actual', 'y_pred'))

In [31]:
show(Scatter(wells, 'LAT', 'LONG', color='PSI'))

In [32]:
wells[['LAT', 'LONG']].describe()

,LAT,LONG
count,11125.000000,11125.000000
mean,35.207878,-96.849979
std,2.689511,7.575851
min,0.000000,-203.625566
25%,34.441536,-97.631560
50%,35.154533,-97.366105
75%,36.124345,-96.537961
max,73.986030,97.695750


In [33]:
wells['LONG'].median()

-97.3661053

In [41]:
## Code adapated from:: http://www.bigendiandata.com/2017-06-27-Mapping_in_Jupyter/

from bokeh.io import output_file, show
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, Range1d, PanTool, WheelZoomTool, BoxSelectTool
)

map_options = GMapOptions(lat=wells['LAT'].median(), lng=wells['LONG'].median(), map_type="roadmap", zoom=6)

plot = GMapPlot(x_range=Range1d(), y_range=Range1d(), map_options=map_options)
plot.title = "Oklahoma Wells"

# For GMaps to function, Google requires you obtain and enable an API key:
#
#     https://developers.google.com/maps/documentation/javascript/get-api-key
#
# Replace the value below with your personal API key:
#plot.api_key = os.getenv('GOOGLE_MAPS_API_KEY')

source = ColumnDataSource(
    data=dict(
        lat=wells['LAT'].values,
        lon=wells['LONG'].values,
    )
)

circle = Circle(x="lon", y="lat", size=15, fill_color="blue", fill_alpha=0.8, line_color=None)
plot.add_glyph(source, circle)

plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())
output_file(join(DATA_DIR, "gmap_plot.html"))
show(plot)

INFO:bokeh.core.state:Session output file './data/gmap_plot.html' already exists, will be overwritten.


In [86]:
wells['County'].unique()

array(['ALFALFA', 'ATOKA', 'BEAVER', 'BECKHAM', 'BLAINE', 'BRYAN',
       'CADDO', 'CANADIAN', 'CARTER', 'CIMARRON', 'CLEVELAND', 'COAL',
       'COMANCHE', 'COTTON', 'CRAIG', 'CREEK', 'CUSTER', 'DEWEY', 'ELLIS',
       'GARFIELD', 'GARVIN', 'GRADY', 'GRANT', 'GREER', 'HARMON',
       'HASKELL', 'HUGHES', 'JACKSON', 'JEFFERSON', 'KAY', 'KINGFISHER',
       'KIOWA', 'LATIMER', 'LEFLORE', 'LINCOLN', 'LOGAN', 'LOVE',
       'MCCLAIN', 'MCINTOSH', 'MAJOR', 'MARSHALL', 'MAYES', 'MURRAY',
       'MUSKOGEE', 'NOBLE', 'NOWATA', 'OKFUSKEE', 'OFUSKEE', 'OKLAHOMA',
       'OKMULGEE', 'PAWNEE', 'PAYNE', 'PITTSBURG', 'PONTOTOC',
       'POTTAWATOMIE', 'ROGER MILLS', 'ROGERS', 'SEMINOLE', 'STEPHENS',
       'TEXAS', 'TILLMAN', 'TULSA', 'WAGONER', 'WASHINGTON', 'WOODS',
       'WOODWARD', nan], dtype=object)

In [87]:
wells.columns

Index(['API#', 'Operator', 'Operator ID', 'WellType', 'WellName', 'WellNumber',
       'OrderNumbers', 'Approval Date', 'County', 'Sec', 'Twp', 'Rng', 'QQQQ',
       'LAT', 'LONG', 'PSI', 'BBLS', 'ZONE'],
      dtype='object')

In [88]:
wells['WellType'].unique()

array(['2R', '2D', 'CDW', 'GS', '2RSI', '2d', nan], dtype=object)

In [89]:
wells['Sec'].unique()

array(['13', '21', '24', '19', '20', '3', '5', '36', '15', '34', '18',
       '10', '17', '8', '26', '30', '32', '28', '2', '29', '27', '14',
       '35', '23', '1', '22', '25', '7', '31', '12', '6', '11', '16', '9',
       '33', '4', '14-12', '09', nan], dtype=object)

In [90]:
wells['Twp'].unique()

array(['28N', '27N', '24N', '25N', '26N', '29N', '16N', '02N', '01N',
       '06N', '05N', '04N', '03N', '10N', '09N', '08N', '11N', '17N',
       '15N', '18N', '14N', '19N', '05S', '06S', '07N', '12N', '13N',
       '02S', '04S', '01S', '03S', '21N', '22N', '23N', '20N', '07S',
       '08S', nan], dtype=object)

In [91]:
wells['Rng'].unique()

array(['10W', '09W', '11W', '12W', '12E', '21E', '26E', '22E', '25E',
       '20E', '24E', '28E', '23E', '27E', '21W', '26W', '25W', '23W',
       '22W', '24W', '13W', '07E', '08E', '05W', '08W', '02W', '01W',
       '03W', '01E', '09E', '04E', '05E', '04W', '10E', '19E', '18E',
       '11E', '17W', '14W', '20W', '19W', '18W', '15W', '16W', '06W',
       '07W', '02E', '03E', '06E', '14E', '15E', '13E', '17E', '16E',
       '13E ', nan], dtype=object)

In [92]:
wells['QQQQ'].unique()

array(['C-SE SE', 'C-NW NW', 'SE NE', ..., 'CT SW SE', 'CT SW SW', nan],
      dtype=object)